# Covid-19 -- Can we gain insights from Unsupervised Learning Models?

**Matt Paterson, hello@HireMattPaterson.com**<br>
**Machine Learning Engineer, Cloud Brigade**<br> 
**Santa Cruz, California**

## Preamble here

import the necessary Python dictionaries

In [1]:
# Import the python libraries that we will use in this project
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.linear_model      import LinearRegression
from sklearn.model_selection   import train_test_split, cross_val_score
from sklearn.metrics           import r2_score, mean_squared_error
from bs4 import BeautifulSoup
import requests

## The Covid Dataset
First we will import the cumulative confirmed cases and deaths by US county as pulled from Google BigQuery's Public Dataset using the following SQL script:
<code>
    SELECT date AS date
      ,subregion1_code AS state_abr  
      ,subregion1_name AS state
      ,subregion2_name AS county
      ,cumulative_confirmed AS confirmed_cases
      ,cumulative_deceased AS deaths
      ,latitude
      ,longitude
      ,average_temperature_celsius
      ,rainfall_mm
      ,subregion2_code AS fips
    FROM `bigquery-public-data.covid19_open_data.covid19_open_data`
    WHERE date = '2020-10-10'
      AND country_name = 'United States of America'
    
    LIMIT 5000;
</code>

## Create a DataFrame
I saved the data set returned by the above sql query into a .csv file. Next I will create a pandas dataframe with that data set so that we can use Python to look it over

In [2]:
path_to_home = '../'
datapath = path_to_home + 'data/'

In [3]:
filename = '2020-10-10_cumulative_covid_data.csv'

covid = pd.read_csv(datapath + filename)
covid.head()

,date,state_abr,state,county,confirmed_cases,deaths,latitude,longitude,average_temperature_celsius,rainfall_mm,fips
0,10/10/2020,VA,Virginia,Waynesboro,326.0,3.0,38.069722,-78.894444,NaN,NaN,51820.0
1,10/10/2020,VA,Virginia,Williamsburg,205.0,8.0,37.270800,-76.706900,NaN,NaN,51830.0
2,10/10/2020,VA,Virginia,Winchester,528.0,4.0,39.178300,-78.166700,NaN,NaN,51840.0
3,10/10/2020,VI,Virgin Islands,NaN,1325.0,20.0,18.333333,-64.833333,NaN,NaN,NaN
4,10/10/2020,VI,Virgin Islands,Saint Croix,489.0,6.0,17.731030,-64.791530,NaN,NaN,78010.0


## The Economics Data Set
I'll use the Beautiful Soup Library from Python to Scrape web pages and compile some data. In the first case, I'm going to go in to wikipedia (note, there is a wiki library for Python but at the moment I am having issues loading it in to my machine).

In [4]:
url = 'https://en.wikipedia.org/wiki/List_of_United_States_counties_by_per_capita_income'
res = requests.get(url)
res.status_code

200

In [5]:
soup = BeautifulSoup(res.content, 'lxml')     # create a BS object for the main page using 'lxml'

In [6]:
soup.contents

['html',
 <html class="client-nojs" dir="ltr" lang="en">
 <head>
 <meta charset="utf-8"/>
 <title>List of United States counties by per capita income - Wikipedia</title>
 <script>document.documentElement.className="client-js";RLCONF={"wgBreakFrames":!1,"wgSeparatorTransformTable":["",""],"wgDigitTransformTable":["",""],"wgDefaultDateFormat":"dmy","wgMonthNames":["","January","February","March","April","May","June","July","August","September","October","November","December"],"wgRequestId":"14058302-0db7-47ec-b8bf-acb3fe63f2ae","wgCSPNonce":!1,"wgCanonicalNamespace":"","wgCanonicalSpecialPageName":!1,"wgNamespaceNumber":0,"wgPageName":"List_of_United_States_counties_by_per_capita_income","wgTitle":"List of United States counties by per capita income","wgCurRevisionId":949141750,"wgRevisionId":949141750,"wgArticleId":45039292,"wgIsArticle":!0,"wgIsRedirect":!1,"wgAction":"view","wgUserName":null,"wgUserGroups":["*"],"wgCategories":["Wikipedia articles in need of updating from July 2019","

In [7]:
table_headings = soup.find_all("th")
table_headings[0].text.split("\n")

['Income in theUnited States of America']

In [8]:
for item in table_headings:
    print(item.text.split("\n")[0])

Income in theUnited States of America
Rank
County or county-equivalent
State, federal district or territory
Per capitaincome
Medianhouseholdincome
Medianfamilyincome
Population
Number ofhouseholds
vteUnited States locations by per capita income
Nationwide
State locations
Federal district
Territory locations
Related lists


In [9]:
table_rows = soup.find_all("tr")
table_rows[9].text.split("\n")

['',
 '1',
 '',
 'New York County',
 '',
 'New York',
 '',
 '$62,498',
 '',
 '$69,659',
 '',
 '$84,627',
 '',
 '1,605,272',
 '',
 '736,192',
 '']

In [10]:
sample = table_rows[3000].text.split("\n")

for item in sample:
    if len(item)<1:
        sample.remove(item)
sample

['2926',
 'Talbot',
 'Georgia',
 '$16,993',
 '$32,424',
 '$38,993',
 '6,689',
 '2,682']

In [11]:
table_rows[3305].text.split("\n")

['',
 '—',
 '',
 'Manu’a District',
 '',
 'American Samoa',
 '',
 '$5,441',
 '',
 '$17,614',
 '',
 '$19,226',
 '',
 '1,143',
 '',
 '282',
 '']

Now I'll take the data structures above and build a pandas dataframe using a for loop and eliminating the empty strings 

In [12]:

# use the table_headings list as the key for a dictionary called incomes
# from index 9 through 3305 inclusive, iterate through the table_rows sublists, build a dictionary with the contents


In [13]:
incomes = {}
headings = []
# for item in table_headings:
#     print(item.text.split("\n")[0])
    
for i in range(1, 9):
    #print(table_headings[i].text.split("\n")[0])
    headings.append(table_headings[i].text.split("\n")[0])
headings

['Rank',
 'County or county-equivalent',
 'State, federal district or territory',
 'Per capitaincome',
 'Medianhouseholdincome',
 'Medianfamilyincome',
 'Population',
 'Number ofhouseholds']

In [37]:
# Function to make dollar amounts in to floating point numbers
def make_float(data):
    '''
    convert dollar valued columns to float
    '''
    if type(data)==float:
        return data
    while "$" in data:
        data = data.replace("$", "")
    while "," in data:
        data = data.replace(",","")
    try:
        #if float(data):
        #    print(f'{data} is a float')
        return float(data)
    except:
        #print(f'{data} is not a float')
        return data

In [44]:
type(int('498'))

int

In [45]:
table_data = []
for i in range(9, 3306):
    sample = []
    sample = table_rows[i].text.split("\n")
    
    for item in sample:
        if len(item)<1:
            sample.remove(item)
    try:
        sample[0] = int(sample[0])
    except:
        sample[0] = 'delete'
    
    if sample[1]=='Virgin Islands' or sample[2]=='American Samoa' or sample[2]=='Puerto Rico':
        sample[0]='delete'
    
    if sample[0]!='delete':
        for i in range(3,8):
            sample[i] = make_float(sample[i])
        table_data.append(sample)
len(table_data)

3143

In [46]:
table_data[1000]

[1001, 'Marshall', 'Kansas', 25262.0, 44032.0, 60625.0, 10057.0, 4345.0]

In [16]:
headings

['Rank',
 'County or county-equivalent',
 'State, federal district or territory',
 'Per capitaincome',
 'Medianhouseholdincome',
 'Medianfamilyincome',
 'Population',
 'Number ofhouseholds']

In [47]:
incomes_df = pd.DataFrame(table_data,columns=['Rank',
 'County or county-equivalent',
 'State, federal district or territory',
 'Per capitaincome',
 'Medianhouseholdincome',
 'Medianfamilyincome',
 'Population',
 'Number ofhouseholds'])
incomes_df.head()

,Rank,County or county-equivalent,"State, federal district or territory",Per capitaincome,Medianhouseholdincome,Medianfamilyincome,Population,Number ofhouseholds
0,1,New York County,New York,62498.0,69659.0,84627.0,1605272.0,736192.0
1,2,Arlington,Virginia,62018.0,103208.0,139244.0,214861.0,94454.0
2,3,Falls Church City,Virginia,59088.0,120000.0,152857.0,12731.0,5020.0
3,4,Marin,California,56791.0,90839.0,117357.0,254643.0,102912.0
4,5,Alexandria City,Virginia,54608.0,85706.0,107511.0,143684.0,65369.0


In [48]:
states_list = incomes_df['State, federal district or territory']

Now that we've created this dataframe, we can save a copy of it in the data folder. Again, this data is scraped from https://en.wikipedia.org/wiki/List_of_United_States_counties_by_per_capita_income and this data, according to wikipedia, 
<quote>
     "is from the 2009-2013 American Community Survey 5-Year Estimates; data for Puerto Rico is from the 2013-2017 American Community Survey 5-Year estimates, and data for the other U.S. territories is from the 2010 U.S. Census.[1][2][3][4] State income levels and income data for the United States as a whole are included for comparison"
</quote>

Note that according to the census bureau's website:<br> 
<t>"Per capita income is the mean income computed for every man, woman, and child in a <t>particular group including those living in group quarters. It is derived by dividing the <t>aggregate income of a particular group by the total population in that group. This measure <t>is rounded to the nearest whole dollar. For the complete definition, go to ACS subject <t>definitions "Income in the Past 12 Months, Per Capita Income."

In [52]:
incomes_df.to_csv(datapath + 'incomes.csv', index=False)

## Create a data table with  racial make-ups as a percentages of population for each county

In the last iteration, we imported racial data from the US Census Bureau. We'll do that here and create a csv of the percentages to use as model inputs.

This demographic data is all found in the US Census Bureau's Website at: https://www.census.gov/data/tables/time-series/demo/popest/2010s-counties-detail.html

and an associated data dictionary is found at https://www2.census.gov/programs-surveys/popest/technical-documentation/file-layouts/2010-2019/cc-est2019-alldata.pdf

In [19]:
filename = 'us_racial.csv'

us_df = pd.read_csv(datapath + filename)
us_df.shape

(716376, 80)

In [20]:
us_demos_df = us_df[(us_df['AGEGRP']==0) & (us_df['YEAR']==12)]
us_demos_df.shape

(3142, 80)

In [21]:
us_demos_df['county_state'] = us_demos_df['CTYNAME'] + ", " + us_demos_df['STNAME']
us_demos_df.shape

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


(3142, 81)

In [22]:
us_demos_df.county_state.head()

209     Autauga County, Alabama
437     Baldwin County, Alabama
665     Barbour County, Alabama
893        Bibb County, Alabama
1121     Blount County, Alabama
Name: county_state, dtype: object

combine columns and divide by population to get a % of each ethnicity

In [24]:

us_demos_df['pct_white'] = (us_demos_df['WAC_MALE'] + us_demos_df['WAC_FEMALE']) / us_demos_df['TOT_POP']
us_demos_df['pct_black'] = (us_demos_df['BAC_MALE'] + us_demos_df['BAC_FEMALE']) / us_demos_df['TOT_POP']
us_demos_df['pct_asian'] = (us_demos_df['AAC_MALE'] + us_demos_df['AAC_FEMALE']) / us_demos_df['TOT_POP']
us_demos_df['pct_hispanic'] = (us_demos_df['H_MALE'] + us_demos_df['H_FEMALE']) / us_demos_df['TOT_POP']
us_demos_df['pct_native_american'] = (us_demos_df['IAC_MALE'] + us_demos_df['IAC_FEMALE']) / us_demos_df['TOT_POP']
us_demos_df['pct_hawaiian'] = (us_demos_df['NAC_MALE'] + us_demos_df['NAC_FEMALE']) / us_demos_df['TOT_POP']

us_demos_df.shape

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in t

(3142, 87)

In [25]:
us_demos_df.columns

Index(['SUMLEV', 'STATE', 'COUNTY', 'STNAME', 'CTYNAME', 'YEAR', 'AGEGRP',
       'TOT_POP', 'TOT_MALE', 'TOT_FEMALE', 'WA_MALE', 'WA_FEMALE', 'BA_MALE',
       'BA_FEMALE', 'IA_MALE', 'IA_FEMALE', 'AA_MALE', 'AA_FEMALE', 'NA_MALE',
       'NA_FEMALE', 'TOM_MALE', 'TOM_FEMALE', 'WAC_MALE', 'WAC_FEMALE',
       'BAC_MALE', 'BAC_FEMALE', 'IAC_MALE', 'IAC_FEMALE', 'AAC_MALE',
       'AAC_FEMALE', 'NAC_MALE', 'NAC_FEMALE', 'NH_MALE', 'NH_FEMALE',
       'NHWA_MALE', 'NHWA_FEMALE', 'NHBA_MALE', 'NHBA_FEMALE', 'NHIA_MALE',
       'NHIA_FEMALE', 'NHAA_MALE', 'NHAA_FEMALE', 'NHNA_MALE', 'NHNA_FEMALE',
       'NHTOM_MALE', 'NHTOM_FEMALE', 'NHWAC_MALE', 'NHWAC_FEMALE',
       'NHBAC_MALE', 'NHBAC_FEMALE', 'NHIAC_MALE', 'NHIAC_FEMALE',
       'NHAAC_MALE', 'NHAAC_FEMALE', 'NHNAC_MALE', 'NHNAC_FEMALE', 'H_MALE',
       'H_FEMALE', 'HWA_MALE', 'HWA_FEMALE', 'HBA_MALE', 'HBA_FEMALE',
       'HIA_MALE', 'HIA_FEMALE', 'HAA_MALE', 'HAA_FEMALE', 'HNA_MALE',
       'HNA_FEMALE', 'HTOM_MALE', 'HTOM_FEMALE

In [27]:
racial_pct = us_demos_df[['county_state','TOT_POP', 'pct_white',
       'pct_black', 'pct_asian', 'pct_hispanic', 'pct_native_american',
       'pct_hawaiian']]

In [28]:
racial_pct.shape

(3142, 8)

In [29]:
racial_pct.head()

,county_state,TOT_POP,pct_white,pct_black,pct_asian,pct_hispanic,pct_native_american,pct_hawaiian
209,"Autauga County, Alabama",55869,0.779735,0.210922,0.017004,0.029909,0.010292,0.001861
437,"Baldwin County, Alabama",223234,0.891903,0.095657,0.015011,0.047188,0.016140,0.001644
665,"Barbour County, Alabama",24686,0.501499,0.490845,0.006157,0.045248,0.012031,0.002471
893,"Bibb County, Alabama",22394,0.777887,0.219121,0.004332,0.027820,0.009244,0.001786
1121,"Blount County, Alabama",57826,0.971310,0.022066,0.005395,0.096531,0.014180,0.001677


Now that we've created this data set we can export it as a csv

In [30]:
racial_pct.to_csv(datapath + 'racial_pct.csv', index=False)